In [ ]:
from minority_report.clean_data import CleanData
from minority_report.scaling import Scaling

import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
from skimage import io, color
from scipy.interpolate import griddata

In [41]:
df = Scaling()
df.load_data()

,precinct_number,offense_type,crime_completed,offense_level,premise_desc,premise,jurisdiction,park_name,suspect_age,suspect_race,suspect_sex,latitude,longitude,metro,victim_age,victim_race,victim_sex,period,borough,patrol_borough
0,73,OFFENSES AGAINST PUBLIC ADMINI,COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,NOT PARK,25-44,BLACK,M,True,True,NOT SUBWAY,<18,BLACK,M,2015-02-11 15:00:00,BROOKLYN,PATROL BORO BKLYN NORTH
1,69,ASSAULT 3 & RELATED OFFENSES,COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,NOT PARK,UNKNOWN,UNKNOWN,UNKNOWN,True,True,NOT SUBWAY,45-64,BLACK,F,2012-03-17 10:00:00,BROOKLYN,PATROL BORO BKLYN SOUTH
2,71,HARRASSMENT 2,COMPLETED,VIOLATION,INSIDE,PUBLIC SCHOOL,N.Y. POLICE DEPT,NOT PARK,<18,BLACK,M,True,True,NOT SUBWAY,18-24,BLACK,M,2016-10-27 13:00:00,BROOKLYN,PATROL BORO BKLYN SOUTH
3,112,RAPE,COMPLETED,FELONY,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,NOT PARK,25-44,WHITE HISPANIC,M,True,True,NOT SUBWAY,25-44,WHITE,F,2014-11-27 19:00:00,QUEENS,PATROL BORO QUEENS NORTH
4,24,GRAND LARCENY,COMPLETED,FELONY,INSIDE,DOCTOR/DENTIST OFFICE,N.Y. POLICE DEPT,NOT PARK,UNKNOWN,UNKNOWN,UNKNOWN,True,True,NOT SUBWAY,45-64,WHITE,F,2013-12-11 13:00:00,MANHATTAN,PATROL BORO MAN NORTH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6983202,109,ASSAULT 3 & RELATED OFFENSES,COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,NOT PARK,25-44,WHITE HISPANIC,M,True,True,NOT SUBWAY,25-44,ASIAN / PACIFIC ISLANDER,F,2018-12-02 04:00:00,QUEENS,PATROL BORO QUEENS NORTH
6983203,25,OFF. AGNST PUB ORD SENSBLTY &,COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,NOT PARK,25-44,BLACK,F,True,True,NOT SUBWAY,25-44,BLACK,M,2018-01-20 00:00:00,MANHATTAN,PATROL BORO MAN NORTH
6983204,41,CRIMINAL MISCHIEF & RELATED OF,COMPLETED,FELONY,FRONT OF,STREET,N.Y. POLICE DEPT,NOT PARK,UNKNOWN,UNKNOWN,UNKNOWN,True,True,NOT SUBWAY,25-44,BLACK HISPANIC,F,2018-08-03 22:00:00,BRONX,PATROL BORO BRONX
6983205,107,FRAUDS,COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,NOT PARK,UNKNOWN,UNKNOWN,U,True,True,NOT SUBWAY,18-24,ASIAN / PACIFIC ISLANDER,M,2018-12-10 11:00:00,QUEENS,PATROL BORO QUEENS SOUTH


In [ ]:
lat_interv = df.data['latitude'].between(40.49611539518921, 40.91553277600008)
lon_interv = df.data['longitude'].between(-74.25559136315213,-73.70000906387347)

In [44]:
df.data['latitude'].min()

False

In [ ]:
print(df['longitude'].min(), df['longitude'].min())

In [19]:
inf = df['period'] > datetime(2016, 2, 1, 0, 0, 0)
sup = df['period'] < datetime(2016, 3, 1, 0, 0, 0)
sample = df[ inf & sup ]

In [20]:
sample.head()

,period,latitude,longitude
66,2016-02-15 15:00:00,40.667652,-73.847544
89,2016-02-07 16:00:00,40.631586,-74.023275
108,2016-02-27 10:00:00,40.651644,-73.913052
145,2016-02-14 13:00:00,40.861855,-73.892459
268,2016-02-13 17:00:00,40.681223,-73.880658


# Test on meshgrid and griddata

In [6]:
X, Y = np.meshgrid([1,2,3], [2,5,6,8])
XI,YI = np.meshgrid([1,2,3],[4,5,6,7])

In [7]:
X

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [8]:
Y

array([[2, 2, 2],
       [5, 5, 5],
       [6, 6, 6],
       [8, 8, 8]])

In [9]:
XI

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [10]:
YI

array([[4, 4, 4],
       [5, 5, 5],
       [6, 6, 6],
       [7, 7, 7]])

In [14]:
grid = griddata((X.ravel(),Y.ravel()), (X**2+Y**2).ravel(), (XI, YI))

In [15]:
grid

array([[19., 22., 27.],
       [26., 29., 34.],
       [37., 40., 45.],
       [51., 54., 59.]])

In [17]:
(X.ravel(),Y.ravel())

(array([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]),
 array([2, 2, 2, 5, 5, 5, 6, 6, 6, 8, 8, 8]))

In [26]:
(X**2+Y**2).ravel()

array([ 5,  8, 13, 26, 29, 34, 37, 40, 45, 65, 68, 73])

# Try out griddata

In [39]:
lat = np.array(sample['latitude'])
lon = np.array(sample['longitude'])
period = np.array(sample['period'])

In [ ]:
values = np.ones((len(lat), len(lon), len(period)), dtype=np.int16)

In [ ]:
image_grid = griddata((lat,lon, period), values, np.zeros(1000, 1000))